# Class Mapping 

|Label | prob | type|
|------|------|-----|
|0     | 0.0  | mono|
|1     | 0.333| mono|
|2     | 0.667| mono|
|3     | 1.0  | mono|
|4     | 0.0  | poly|
|5     | 0.333| poly|
|6     | 0.667| poly|
|7     | 1.0  | poly|

In [1]:
from utils.classifier import *
from utils.dataloader import *
from utils.features import *
from utils.helper import *
from utils.processing import *
import pickle

In [2]:
RANDOM_STATE = 66
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
PARAMS = []
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [3]:
from sklearn.model_selection import train_test_split
img_path, prob, types, labels= load_data(DATA_PATH)

feature_builder = FeatureExtraction(DATA_PATH, img_path, labels)
X_train, X_test, y_train, y_test = feature_builder.split_data(train_test_split, randome_state=RANDOM_STATE, stratify=True, split_ratio=0.25)

[0 1 2 3 4 5 6 7] [588 117  56 313 920 178  50 402]


In [4]:
from sklearn.metrics import make_scorer, f1_score, balanced_accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# autoAug 
# Genearte more 
augment_funcs = [
    # flip_x,
    # flip_y,
    # rotate,
    salt_pepper_noise,
]

preprocess_pipeline = [
    (strech_img, {
        }),
    # (clach_img, {
    #     'clipLimit': 2.0,
    #     'tileGridSize': (8, 8)}),
    # (guassian_blur, {
    #     'kernel_size': (0, 0),
    #     'sigmaX': 3}),
    # (lap_feature, {
    #     'dst': -1,
    #     'ksize' : 5}),
    # (morpo_opening, {
    #     'kernel': np.ones((2,2), np.uint8),
    #     'iterations': 1}),
    # (morph_smoothing, {
    # 'ses' : feature_builder.ses})
]

# Smaller cell: more features
# 
hog_params = {
    'orient': 10,
    'pix_per_cell': 10,
    'cell_per_block': 1,
    'vis': False,
    'block_norm': 'L2'
}

svm_param = {
    'C': 10,
    'kernel' : 'rbf',
    # 'gamma' : 0.0001,
    # 'class_weight' : 'balanced',
    # 'verbose' : False,
}

grid_svm_params = {
    'estimator': SVC,
    'model_params': {
        'kernel': ['rbf', 'linear'],
        'C' : [0.1, 0.3, 0.5, 0.7, 1, 3, 5, 7 ,9, 10],
        'class_weight': ['balanced']},
    'scoring': {
        "F1_Weighted": make_scorer(f1_score, average='weighted'),
        "Balanced_Accuracy": make_scorer(balanced_accuracy_score),},
    'refit': 'Balanced_Accuracy',
    'verbose' : 3
}

smote_params = {
    'total_classes': np.unique(y_train),
    'grid_param': grid_svm_params,
    'knn' : KNeighborsClassifier,
    'n_neighbors': 5,
}

PARAMS.append(augment_funcs)
PARAMS.append(preprocess_pipeline)
PARAMS.append(hog_params)
# PARAMS.append(svm_param)
# PARAMS.append(grid_svm_params)
PARAMS.append(smote_params)

In [5]:
X_train_aug, y_train_aug = feature_builder.augmentation(X_train, y_train, augment_funcs=augment_funcs)

Augmenting images: 100%|██████████| 1968/1968 [00:00<00:00, 5767.47it/s]

Augmenting done, added image: 300


In [6]:
X_train_aug, y_train_aug = feature_builder.augmentation(X_train, y_train, augment_funcs=augment_funcs)

X_train_prepro = feature_builder.preprocess(X_train_aug, preprocess_pipeline = preprocess_pipeline)
X_test_prepro = feature_builder.preprocess(X_test, preprocess_pipeline = preprocess_pipeline)

print(X_train_prepro.shape, y_train_aug.shape)
print(X_test_prepro.shape, y_test.shape)

Augmenting images: 100%|██████████| 1968/1968 [00:00<00:00, 6048.74it/s]

Augmenting done, added image: 300



Pre-processing images:  76%|███████▌  | 1717/2268 [09:33<03:49,  2.40it/s]

In [ ]:
X_train_prepro,  X_test_prepro = None, None

with open(os.path.join(OUT_PATH ,'X_morph_smoothing.pkl'), 'rb') as f:
    X_train_prepro, X_test_prepro = pickle.load(f)

In [ ]:
train_hog_des = feature_builder.get_hog_features(X_train_prepro, **hog_params)
test_hog_des = feature_builder.get_hog_features(X_test_prepro, **hog_params)

print(train_hog_des.shape)
print(test_hog_des.shape)

Calculating descriptors: 100%|██████████| 656/656 [00:04<00:00, 132.87it/s]

(1968, 9000)
(656, 9000)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(train_hog_des)
train_X = scaler.transform(train_hog_des)
test_X = scaler.transform(test_hog_des)  

In [ ]:
result, best_clf = grid_search(train_X, y_train_aug, **grid_svm_params)
grid_pred = best_clf.predict(test_X)
get_report(y_test, grid_pred)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 3/3] END C=0.1, class_weight=balanced, kernel=linear; Balanced_Accuracy: (train=1.000, test=0.474) F1_Weighted: (train=1.000, test=0.702) total time=  36.0s
[CV 1/3] END C=0.1, class_weight=balanced, kernel=linear; Balanced_Accuracy: (train=1.000, test=0.444) F1_Weighted: (train=1.000, test=0.670) total time=  36.7s
[CV 2/3] END C=0.1, class_weight=balanced, kernel=linear; Balanced_Accuracy: (train=1.000, test=0.473) F1_Weighted: (train=1.000, test=0.701) total time=  37.2s
[CV 2/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.853, test=0.418) F1_Weighted: (train=0.801, test=0.607) total time=  51.4s
[CV 1/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.846, test=0.430) F1_Weighted: (train=0.808, test=0.600) total time=  51.6s
[CV 1/3] END C=0.1, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.391, test=0.356) F1_Weighted: (train=0.125, test=0.121) t

In [10]:
smote_results, smote_clfs = one_vs_other_up_sampling(train_X, y_train_aug, **smote_params)
smote_distance, smote_preds = distance_vote(test_X, smote_clfs)
get_report(y_test, smote_preds)

Generated 1323 synthetic samples.
Trainig on: (array([-1.,  0.]), array([1527, 1764]))
Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END C=0.1, class_weight=balanced, kernel=linear; Balanced_Accuracy: (train=1.000, test=0.960) F1_Weighted: (train=1.000, test=0.962) total time=  23.3s
[CV 3/3] END C=0.1, class_weight=balanced, kernel=linear; Balanced_Accuracy: (train=1.000, test=0.957) F1_Weighted: (train=1.000, test=0.960) total time=  25.4s
[CV 2/3] END C=0.1, class_weight=balanced, kernel=linear; Balanced_Accuracy: (train=1.000, test=0.959) F1_Weighted: (train=1.000, test=0.962) total time=  25.4s
[CV 1/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.965, test=0.929) F1_Weighted: (train=0.968, test=0.932) total time=  27.7s
[CV 2/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.941, test=0.932) F1_Weighted: (train=0.944, test=0.937) total time=  33.5s
[CV 1/3] END C=0.1, class_weight=balanced, kernel=rbf